<a href="https://colab.research.google.com/github/chun-ming-Lin/yolov4_mask/blob/master/colab/YOLOv4_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

Fri Aug 13 06:46:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 取得訓練資料

In [ ]:
# 之前從kaggle上抓到的dataset
# 從google drive複製過來
!cp '/content/drive/MyDrive/YOLOv4/YOLOv4_mask/medical-masks-dataset.zip' '/content/'
!unzip '/content/medical-masks-dataset.zip' -d '/content/mask_data/'

Archive:  /content/medical-masks-dataset.zip
  inflating: /content/mask_data/images/-1x-1.jpg  
  inflating: /content/mask_data/images/-I1-MS09uaqsLdGTFkgnS0Rcg1mmPyAj95ySg_eckoM.jpeg  
  inflating: /content/mask_data/images/0002526673.jpg  
  inflating: /content/mask_data/images/0009S6815V3PEU1N-C123-F4.jpg  
  inflating: /content/mask_data/images/000_1OC3DT.jpg  
  inflating: /content/mask_data/images/000_1ov3n5_0.jpeg  
  inflating: /content/mask_data/images/000b7b75-1600.jpg  
  inflating: /content/mask_data/images/001_1024.jpeg  
  inflating: /content/mask_data/images/002_1024.jpeg  
  inflating: /content/mask_data/images/003_1024.jpeg  
  inflating: /content/mask_data/images/004_1024.jpeg  
  inflating: /content/mask_data/images/0109-00176-096b1.jpg  
  inflating: /content/mask_data/images/012106.jpg_1140x855.jpg  
  inflating: /content/mask_data/images/012420_coronoa_masks_web.jpg  
  inflating: /content/mask_data/images/0200b38c89b16c37c5de8e247bb00c2f.jpg  
  inflating: /conte

In [ ]:
# %cd ..
# %cd mask_data/
# !cat labels/-1x-1.xml

In [ ]:
from bs4 import BeautifulSoup
import os
import shutil
classes_dict = {'good':2, 'bad':1, 'none':0}

def Convert2YoloFormat(filepath, labelpath, yolo_labels_path):
  with open(filepath, 'r') as f:
    soup = BeautifulSoup(f.read(), 'xml')
    filename_org = soup.select_one('filename').text
    filename_idx = filename_org.rfind('.')
    filename = filename_org[:filename_idx]
    width = int(soup.select_one('width').text)
    height = int(soup.select_one('height').text)
    arg = []
    for obj in soup.select('object'):
      classes = obj.select_one('name').text
      xmin = int(obj.select_one('xmin').text)
      ymin = int(obj.select_one('ymin').text)
      xmax = int(obj.select_one('xmax').text)
      ymax = int(obj.select_one('ymax').text)
      x = ((xmin + xmax) / 2) / width
      y = ((ymin + ymax) / 2) / height
      w = (xmax - xmin) / width
      h = (ymax - ymin) / height
      obj_class = classes_dict.get(classes)
      arg.append(' '.join([str(obj_class), str(x), str(y), str(w), str(h)]))
  
  with open(os.path.join(yolo_labels_path, filename+'.txt'), 'w') as outfile:
    outfile.write('\n'.join(arg))
    

In [ ]:
yolo_labels_path = '/content/mask_data/yolo_labels'
labelpath = '/content/mask_data/labels'
imagepath = '/content/mask_data/images'
if not os.path.exists(yolo_labels_path):
  os.makedirs(yolo_labels_path)

bad_data = []

for label in os.listdir(labelpath):
  filepath = os.path.join(labelpath, label)
  try:
    Convert2YoloFormat(filepath, labelpath, yolo_labels_path)
  except Exception as e:
    print(e)
    bad_data.append(filepath)
bad_data

float division by zero


['/content/mask_data/labels/83855_1580055989W0WA.xml']

In [ ]:
# remove bad images
for data in bad_data:
  path = os.path.join(imagepath, data)
  if os.path.isfile(path):
    os.remove(path)

# Create train.txt

In [ ]:
# copy images file to labels path
img_file = []
for filename in os.listdir('/content/mask_data/images'):
  source = os.path.join(imagepath, filename)
  destination = os.path.join(yolo_labels_path, filename)
  shutil.copyfile(source, destination)
  img_file.append(destination)

with open('/content/mask_data/train.txt', 'w') as outfile:
  for img in img_file:
    outfile.write(img)
    outfile.write('\n')
  outfile.close()

  

#複製DARKNET

In [ ]:
! git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15298, done.
remote: Total 15298 (delta 0), reused 0 (delta 0), pack-reused 15298
Receiving objects: 100% (15298/15298), 13.67 MiB | 23.49 MiB/s, done.
Resolving deltas: 100% (10400/10400), done.


#修改 Makefile
* GPU=1
* CUDNN=1
* OPENCV=1

In [ ]:
! sed -i "s/GPU=0/GPU=1/g" /content/darknet/Makefile
! sed -i "s/CUDNN=0/CUDNN=1/g" /content/darknet/Makefile
! sed -i "s/OPENCV=0/OPENCV=1/g" /content/darknet/Makefile

In [ ]:
! head /content/darknet/Makefile

GPU=1
CUDNN=1
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=0
ZED_CAMERA=0
ZED_CAMERA_v2_8=0



#編譯 **YOLO**

In [ ]:
!cd darknet; make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_

In [ ]:
!cd darknet; ./darknet

usage: ./darknet <function>


#修改.cfg檔
* batch = 64
* subdivisions = 16
* [yolo] 區塊的classes = 3  (有三個)
* [yolo] 區塊的前一個 [convolution] 區塊的 filter  = 24 (有三個)

* YOLOV4 偵測的濾鏡(filter) 大小為 (C+5)*B.
  * B 是每個Feature Map 可以偵測的Bounding Box 數量
  * 5 是Bounding Box 的x,y,w,h 以及 Confidence
  * C 是類別數量
* filters=(classes + 5)*3 # 24
* classes=3

## 複製 .data .names .cfg .weights

In [ ]:
!cp '/content/drive/MyDrive/YOLOv4/YOLOv4_mask/obj.data' '/content/mask_data'
!cp '/content/drive/MyDrive/YOLOv4/YOLOv4_mask/obj.names' '/content/mask_data'
!cp '/content/drive/MyDrive/YOLOv4/YOLOv4_mask/yolov4-mask.cfg' '/content/mask_data'
# !cp '/content/drive/MyDrive/YOLOv4/yolov4.conv.137' '/content/mask_data'

In [ ]:
!cp '/content/drive/MyDrive/YOLOv4/YOLOv4_mask/backup/yolov4-mask_1000.weights' '/content/mask_data/yolov4-mask_1000.weights'

# Train 開始訓練

In [ ]:
!cd darknet;! ./darknet detector train /content/mask_data/obj.data /content/mask_data/yolov4-mask.cfg /content/mask_data/yolov4-mask_1000.weights -dont_show -map

串流輸出內容已截斷至最後 5000 行。
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.808819), count: 16, class_loss = 5.097096, iou_loss = 16.019245, total_loss = 21.116341 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.752118), count: 3, class_loss = 0.003483, iou_loss = 0.234080, total_loss = 0.237563 
 total_bbox = 1330632, rewritten_bbox = 0.028332 % 
Failed to infer label file name (check image extension is supported): /content/mask_data/yolo_labels/1197315184.jpg.0.jpg 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 139 Avg (IOU: 0.757028), count: 23, class_loss = 3.977857, iou_loss = 69.215042, total_loss = 73.192902 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 150 Avg (IOU: 0.812665), count: 17, class_loss = 1.208921, iou_loss = 18.948755, total_loss = 20.157677 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 161 Avg (IOU: 0.778377), count: 2, class_loss = 0.40

## 模擬點擊
Chrome -> 右鍵檢查 -> Console -> clear -> 貼上以下的Code

```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```

# Test

In [ ]:
# !cp '/content/drive/MyDrive/backup/yolov4-mask_last.weights' '/content/yolov4-mask_last.weights'

In [ ]:
# !cd darknet; ! ./darknet detector test /content/cfg_mask/obj.data /content/cfg_mask/yolov4-mask.cfg /content/yolov4-mask_last.weights /content/yolo/0.jpg

In [ ]:
# !cp '/content/cfg_mask/yolov4-mask.cfg' '/content/drive/MyDrive/Colab Notebooks/ai_dataset/yolov4-mask.cfg'